In [17]:
import numpy as np
import pandas as pd
from groq import Groq
import google.generativeai as genai
import editdistance

import time
import nltk
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')

from mistralai import Mistral
import anthropic
from openai import OpenAI
import time
import warnings
import google.generativeai as genai



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/prateekchhikara/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [18]:
def sentancePicker(flag,dataSet):
    if flag==0:
        val=20
    elif flag==1:
        val=50
    else:
        val=100

    tempSet=set()
    n=len(dataSet)

    while(len(tempSet)<val):
        idx=np.random.randint(0,n)
        tempSet.add(dataSet[idx][0])
    
    return list(tempSet)
    

In [ ]:
def preprocessing():
    data=pd.read_csv('dsbanglore.csv', sep='delimiter', header=None ,engine='python')
    data=np.array(data)
    n=data.shape[0]
    list=[]
    for i in range(n):
        list.append(data[i]) 
    return list

(625, 1)


In [4]:
def find(list_original, item) :
    text = word_tokenize(item)
    tagged = nltk.pos_tag(text)
    # print(tagged)
    imp = set()
    for each in tagged :
        if(each[1] == 'JJ' or each[1] == 'JJR' or each[1] == 'JJS' or each[1] == 'NN' or each[1] == 'NNS' or each[1] == 'NNP' or each[1] == 'NNPS' or each[1] == 'VB' or each[1] == 'VBD' or each[1] == 'VBG' or each[1] == 'VBN' or each[1] == 'VBP' or each[1] == 'VBZ') :
            imp.add(each[0])
    
    max = 0 
    final = ""
    for each_org in list_original :
        count = 0 
        for each in imp :
            if each in each_org :
                count += 1
        if(count > max) :
            max = count 
            final = each_org
    
    return final 


In [5]:
def editDistance(list_original, list_llm) :

    error = 30
    list_ans = set()

    for item in list_llm :
        if(len(item) > 4 and (item[1] == '.' or item[2] == "." or item[3] == "." or item[4] == "." or item[1] == ':' or item[2] == ":" or item[3] == ":" or item[4] == ":")) :
            updated_item = item[item.index('.')+1:].strip()
            # print(updated_item)

            if(updated_item not in list_original) :

                edit_distance = 10000
                edit_string = ""
                for org_item in list_original :
                    # print(org_item)
                    present_dist = editdistance.eval(updated_item, org_item)

                    if present_dist < edit_distance :
                        edit_distance = present_dist
                        edit_string = org_item

                if(edit_distance < error) :
                    list_ans.add(edit_string) 
                    print(updated_item , "------>", edit_string, "------->", edit_distance) #, len(updated_item))
                else :
                    similar = find(list_original, updated_item)
                    if(len(similar) > 0) :
                      list_ans.add(similar)
                      print(updated_item , "------>", similar)

            else :
                list_ans.add(updated_item) 

    return list_ans

In [ ]:
def model_claude(model_name, num_tokens, sp, up):

    claude_client = anthropic.Anthropic(api_key = "")
    while True : 
        try :
            message = claude_client.messages.create(
                    model=model_name,
                    max_tokens=num_tokens,
                    temperature=0,
                    top_p = 1,
                    system= sp,
                    messages=[{"role": "user", "content": up}]
            )
            break 

        except Exception as e:
            print(model_name + " went into exception ....", e)
            print("Sleeping for 2 minutes")
            time.sleep(120) 

    
    return message.content[0].text.split("\n")

In [ ]:
def llama_model(num_tokens , sp , up):

    
    client = Groq(api_key = "")
    while True : 
        try :
            completion = client.chat.completions.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[{"role": "system", "content": sp},{"role": "user", "content": up}],
                temperature=0,
                max_completion_tokens=num_tokens,
                top_p=1,
                stream=False,
                stop=None,
            )
            break 
        except Exception as e:
            print("llama went into exception ....", e)
            print("Sleeping for 2 minutes")
            time.sleep(70) 

    print(completion.choices[0].message.content)
    return completion.choices[0].message.content.split("\n")

In [ ]:
def model_mixtral(num_tokens, sp, up) :
    mixtral_client = Mistral(api_key="")
    while True : 
        try :

            completion = mixtral_client.chat.complete(
                        model = "open-mistral-nemo",
                        messages=[{"role": "system", "content": sp},{"role": "user", "content": up}],
                        temperature=0,
                        max_tokens=num_tokens,
                        top_p=1.0,
            )

            break 

        except Exception as e:
            print("open-mistral-nemo went into exception ....", e)
            print("Sleeping for 2 minutes")
            time.sleep(70) 

    print(completion.choices[0].message.content)
    return completion.choices[0].message.content.split("\n")

In [ ]:
def gemmini(list , max_token , system_prompt, user_prompt):
    genai.configure(api_key="")

    res_str = ""
    for i,each_sent in enumerate(list) :
        res_str += str(i+1) + ". " + each_sent + "\n" ;


    generation_config = {
      "temperature": 0,
      "top_p": 1,
      "top_k": 1,
      "max_output_tokens": max_token,
      "response_mime_type": "text/plain",
    }
   
    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-lite-preview-02-05",
      generation_config=generation_config,
      system_instruction=system_prompt,
    )
    
    chat_session = model.start_chat(
      history=[
        {
          "role": "user",
          "parts": [user_prompt],
       
        },
      ]
    ) 
    
    response = chat_session.send_message(res_str)
    response=response.text
    ans=response.strip().split("\n")
    return ans


In [1]:
mainDict={}
def main():
    
    processedData=preprocessing() 

    for i in range(9):
        size=0
        selected_elements=0
        list=[]
        if (i<=2):
            list=sentancePicker(0,processedData)
            selected_elements=10            
            size=20
        elif ( i>=3 and i<=5):
            list=sentancePicker(1,processedData)
            selected_elements=25
            size=50
        else :
            list=sentancePicker(2, processedData)
            selected_elements=50
            size=100
            
        dict={}
        for j in range(len(list)):
            tempList=  [0,0,0,0,0,0,0,0,0,0,0]
            dict[list[j]]=tempList
            
            
        res = ""
        for j,each_sent in enumerate(list) :
            res += str(j+1) + ". " + each_sent + "\n" ;

        for j in range(10):

            system_instruction= "Select best "+ str(selected_elements)+ "sentences that summarizes the input text. Output the sentence along with the sentence number that best summarizes the above input. Think step by step and follow the instructions."
            user_instruction  = "Input consist of " + str(size) + " sentences. Each sentence is present in a new line. Each sentence contains a sentence number followed by text. You are an assistant that selects best "+ str(selected_elements)+ " sentences (subset) which summarizes the input. Note that - summary should not contain duplicates and there should be no paraphrasing.\n\n"
            outputList=[]
           
            if(i%3==0):
                outputList=gemmini(list ,8000,system_instruction , user_instruction)
            elif(i%3==1):
                outputList=model_mixtral(8000,system_instruction , user_instruction+res)
            else:
                outputList=llama_model(8000,system_instruction,user_instruction+res)

     
            finalList=editDistance(list , outputList)

            for k in finalList:
                    dict[k][j]+=1
            
        modelName=""
        # total probability ke leye
        if(i%3==0):
            modelName="gemini"
        elif(i%3==1):
            modelName="mixtral"
        else:
            modelName="llama"

        for each in list:
            dict[each][10]=0
            for k in range(10):
                dict[each][10]=dict[each][10]+dict[each][k]
        # print(i)
        # print(modelName)
        keyList=[]
        valList=[]
        for vals in dict:
            keyList.append(vals)
            valList.append(dict.get(vals))
        keyList=np.array(keyList)
        keyList=keyList.reshape(-1,1)
        valList=np.array(valList)
        
        fans=np.hstack((keyList,valList))
        fans=pd.DataFrame(fans)
        fans.to_excel(modelName+'_'+str(size)+""+'.xlsx')

In [ ]:
main()